# TRABALHO 03 - DETECÇÃO

Alunos: Alani Rigotti, Ari Júnior, Felipe Araújo e Luigi Marchetti

# Bibliotecas

**Ultralytics** é uma biblioteca voltada para detecção de objetos em vídeo e imagem. Foi utilizado os modelos yolo11n.pt, yolo11x.pt, yolo11m.pt e yolov8x.pt.

`model = YOLO("yolo11n.pt")`



In [ ]:
from ultralytics import YOLO

**CV** é uma biblioteca que fornece as ferramentas para o processamento de imagens e análise de vídeo.

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

# Modelo escolhido

A máquina utilizada possui um processador Ryzen 5 5600 com clock de 3.5 GHz e placa gráfica RTX 3060 com 3584 CUDA Cores

| Modelo       | FPS | Ano  |
|--------------|-----|------|
| YOLO11n      | 20  | 2024 |
| YOLO11m      | 5   | 2024 |
| YOLO11x      | 2   | 2024 |
| YOLOv8x      | 2   | 2023 |

In [ ]:
# Load the YOLO model
model = YOLO("yolo11n.pt")
model = YOLO("yolov8x.pt")
model = YOLO("yolo11x.pt")
model = YOLO("yolo11m.pt")

# Vídeo

Os vídeos foram retirados do site **MOTChallenge**, ele disponibiliza um grande coleção de conjuntos de dados para a comunidade dev.

In [ ]:
# Open the video file
video_path = "test2.mp4"
cap = cv2.VideoCapture(video_path)

paused = False  # Flag de pausa

# Algoritmo V1

In [ ]:
# Loop principal: continua enquanto o vídeo estiver aberto
while cap.isOpened():

    # Só processa o próximo frame se não estiver pausado
    if not paused:
        # Lê o próximo frame do vídeo
        success, frame = cap.read()
        if not success:
            break  # Sai do loop se não conseguir ler o frame (fim do vídeo ou erro)

        # Obtém o timestamp atual em milissegundos
        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
        total_ms = int(timestamp_ms)

        # Converte o timestamp para o formato MM:SS.mmm
        minutes = (total_ms // 1000) // 60
        seconds = (total_ms // 1000) % 60
        milliseconds = total_ms % 1000
        timestamp_str = f"{minutes:02d}:{seconds:02d}.{milliseconds:03d}"

        # Obtém o número do frame atual
        frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        # Aplica o modelo YOLO para detecção de objetos no frame
        results = model(frame, verbose=False)
        boxes = results[0].boxes         # Caixas delimitadoras detectadas
        names = model.names              # Nomes das classes reconhecidas pelo modelo
        target_classes = ["person", "car"]  # Classes de interesse
        count = {"person": 0, "car": 0}     # Contadores de objetos detectados

        # Loop sobre cada caixa detectada
        for box in boxes:
            cls_id = int(box.cls[0])            # ID da classe detectada
            class_name = names[cls_id]          # Nome da classe
            if class_name in target_classes:    # Se for pessoa ou carro
                count[class_name] += 1          # Incrementa contador

                # Obtém coordenadas da caixa delimitadora
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                conf = float(box.conf[0])       # Confiança da detecção
                label = f"{class_name} {conf:.2f}"

                # Cor da caixa: verde para pessoa, azul para carro
                color = (0, 255, 0) if class_name == "person" else (255, 0, 0)

                # Desenha a caixa e o rótulo no frame
                cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 2)
                cv2.putText(frame, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Exibe no terminal o timestamp, número do frame e contagem de pessoas e carros detectados
        print(f"[{timestamp_str} | Frame {frame_id}] Pessoas: {count['person']}, Carros: {count['car']}")

    # Mostra o frame atual na janela (mesmo quando pausado)
    cv2_imshow(frame)

    # Aguarda uma tecla ser pressionada por 1ms
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):        # Tecla 'q' para sair
        break
    elif key == ord(' '):      # Tecla espaço para pausar/despausar
        paused = not paused

# Libera recursos ao final do vídeo
cap.release()
cv2.destroyAllWindows()


# Algoritmo como parâmetros

In [ ]:
# Parâmetros para contagem e verificação de duplicatas
MIN_PERSIST_TIME_MS = 1000  # tempo mínimo para contar (em ms)
DUPLICATE_DISTANCE_THRESHOLD = 50  # distância mínima para considerar duplicado (em pixels)
DUPLICATE_TIME_THRESHOLD_MS = 1000  # tempo mínimo entre objetos próximos


# Dicionários para armazenar histórico e IDs já contados
track_history = {}
counted_ids = {"person": set(), "car": set()}
last_positions = {"person": [], "car": []}  # posições recentes para verificar duplicação

# Função auxiliar para obter o centro da caixa delimitadora (bounding box)
def get_center(xyxy):
    x1, y1, x2, y2 = xyxy
    return ((x1 + x2) // 2, (y1 + y2) // 2)

# Loop principal para processar os frames do vídeo
while cap.isOpened():
    if not paused:
        success, frame = cap.read()
        if not success:
            break  # Encerra se não houver mais frames

        # Obtém timestamp e informações do frame atual
        timestamp_ms = int(cap.get(cv2.CAP_PROP_POS_MSEC))
        minutes = (timestamp_ms // 1000) // 60
        seconds = (timestamp_ms // 1000) % 60
        milliseconds = timestamp_ms % 1000
        timestamp_str = f"{minutes:02d}:{seconds:02d}.{milliseconds:03d}"
        frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        # Faz a inferência com o modelo YOLO com tracking
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes
        names = model.names
        target_classes = ["person", "car"]
        current_count = {"person": 0, "car": 0}  # contagem atual no frame

        # Se houver caixas com IDs (tracking ativo)
        if boxes.id is not None:
            for i in range(len(boxes.cls)):
                cls_id = int(boxes.cls[i])
                track_id = int(boxes.id[i])
                class_name = names[cls_id]

                # Pula classes que não queremos contar
                if class_name not in target_classes:
                    continue

                # Obtém a posição e o centro da caixa
                xyxy = boxes.xyxy[i].cpu().numpy().astype(int)
                center = get_center(xyxy)

                # Atualiza o histórico de rastreamento
                if track_id not in track_history:
                    track_history[track_id] = {
                        "class": class_name,
                        "first_seen": timestamp_ms,
                        "last_seen": timestamp_ms,
                        "center": center,
                        "counted": False,
                    }
                else:
                    track_history[track_id]["last_seen"] = timestamp_ms
                    track_history[track_id]["center"] = center

                # Verifica se o objeto deve ser contado
                track = track_history[track_id]
                duration = track["last_seen"] - track["first_seen"]

                if not track["counted"] and duration >= MIN_PERSIST_TIME_MS:
                    # Verifica se é uma duplicata próxima
                    too_close = False
                    for prev_id, prev_center, prev_time in last_positions[class_name]:
                        dist = np.linalg.norm(np.array(center) - np.array(prev_center))
                        time_diff = timestamp_ms - prev_time
                        if dist < DUPLICATE_DISTANCE_THRESHOLD and time_diff < DUPLICATE_TIME_THRESHOLD_MS:
                            too_close = True
                            break

                    # Se não for duplicado, conta o objeto
                    if not too_close:
                        counted_ids[class_name].add(track_id)
                        last_positions[class_name].append((track_id, center, timestamp_ms))
                        track["counted"] = True

                # Se o objeto já foi contado, desenha no frame e atualiza contagem
                if track["counted"]:
                    current_count[class_name] += 1
                    label = f"{class_name} ID:{track_id}"
                    color = (0, 255, 0) if class_name == "person" else (255, 0, 0)
                    cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 2)
                    cv2.putText(frame, label, (xyxy[0], xyxy[1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Total de pessoas e carros únicos já contados
        total_persons = len(counted_ids["person"])
        total_cars = len(counted_ids["car"])

        # Exibe as informações no terminal
        print(f"[{timestamp_str} | Frame {frame_id}] Pessoas ativas: {current_count['person']} (total únicas: {total_persons}), Carros ativos: {current_count['car']} (total únicos: {total_cars})")

    # Mostra o frame com as detecções
    cv2.imshow("YOLO + Tracking + Contagem Robusta", frame)

    # Comandos do teclado: 'q' para sair, espaço para pausar/resumir
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord(' '):
        paused = not paused

# Libera os recursos ao final
cap.release()
cv2.destroyAllWindows()
